# **Cocoon: Model your Databases**


## **You Need...**

1. LLM API (cost is typically <50 cents per table)
2. Data Warehouse Con (Snowflake/DuckDB/csv)



In [ ]:
! pip install cocoon_data==0.1.121

In [1]:
from cocoon_data import *

In [2]:
#@title  Download example table (skip this step if you have your own table)
import requests
import os
import base64

def download_github_directory(repo_owner, repo_name, directory_path, local_path):
    # GitHub API endpoint
    api_url = f"https://api.github.com/repos/{repo_owner}/{repo_name}/contents/{directory_path}"

    # Send a GET request to the GitHub API
    response = requests.get(api_url)

    # Check if the request was successful
    if response.status_code == 200:
        # Create the local directory if it doesn't exist
        os.makedirs(local_path, exist_ok=True)

        # Parse the JSON response
        contents = response.json()

        # Loop through each file in the directory
        for item in contents:
            if item['type'] == 'file':
                # Get the file content
                file_content = requests.get(item['download_url']).content

                # Save the file locally
                file_path = os.path.join(local_path, item['name'])
                with open(file_path, 'wb') as file:
                    file.write(file_content)
                print(f"Downloaded: {item['name']}")
            elif item['type'] == 'dir':
                # If it's a subdirectory, recursively download its contents
                subdir_path = os.path.join(directory_path, item['name'])
                local_subdir_path = os.path.join(local_path, item['name'])
                download_github_directory(repo_owner, repo_name, subdir_path, local_subdir_path)
    else:
        print(f"Failed to retrieve directory contents. Status code: {response.status_code}")

# Usage
repo_owner = "Cocoon-Data-Transformation"
repo_name = "cocoon"
directory_path = "documentation/model_gallery/linkedin"
dbt_directory = "./linkedin"
dbt_name = "Linkedin"

download_github_directory(repo_owner, repo_name, directory_path, dbt_directory)

Downloaded: cocoon_er.yml
Downloaded: cocoon_join.yml
Downloaded: model.html
Downloaded: snapshot_linkedin_account_history_data.sql
Downloaded: snapshot_linkedin_account_history_data.yml
Downloaded: snapshot_linkedin_campaign_group_history_data.sql
Downloaded: snapshot_linkedin_campaign_group_history_data.yml
Downloaded: snapshot_linkedin_campaign_history_data.sql
Downloaded: snapshot_linkedin_campaign_history_data.yml
Downloaded: snapshot_linkedin_creative_history_data.sql
Downloaded: snapshot_linkedin_creative_history_data.yml
Downloaded: sources.yml
Downloaded: stg_linkedin_account_history_data.html
Downloaded: stg_linkedin_account_history_data.sql
Downloaded: stg_linkedin_account_history_data.yml
Downloaded: stg_linkedin_ad_analytics_by_campaign_data.html
Downloaded: stg_linkedin_ad_analytics_by_campaign_data.sql
Downloaded: stg_linkedin_ad_analytics_by_campaign_data.yml
Downloaded: stg_linkedin_ad_analytics_by_creative_data.html
Downloaded: stg_linkedin_ad_analytics_by_creative_da

In [3]:
#@title Provide your LLM API

# if you use Open AI, please ensure GPT-4 is available
# openai.api_key  = ''

# # if you use anthropic, please ensure Claude 3 is available
os.environ["ANTHROPIC_API_KEY"] = ""
openai.api_type ='Anthropic'

# # if you use Vertex AI, please ensure Claude 3 is available
# openai.api_type = 'AnthropicVertex'
# os.environ['AnthropicVertex_region'] = "us-east5"
# os.environ['AnthropicVertex_project_id'] = ""

# test if LLM works
test_message = "hello"
messages = [{"role": "user", "content": test_message}]
response = call_llm_chat(messages, temperature=0.1, top_p=0.1)
print(response['choices'][0]['message']['content'])

Hello! How can I assist you today?


In [4]:
# @title Provide Data Warehouse Con

# In-mem duckdb loads the df
con = duckdb.connect(database=':memory:')

seeds_directory = dbt_directory + "/seeds/"
files = os.listdir(seeds_directory)

for file in files:
    if file.endswith(".csv"):
        file_path = f"{seeds_directory}/{file}"
        table_name = file.split(".")[0]
        table_name = clean_table_name(table_name)
        # Read the CSV file with pandas
        df = pd.read_csv(file_path)
        df.columns = [clean_column_name(col) for col in df.columns]
        # Use SQL to create a persistent table from the registered DataFrame
        con.execute(f"CREATE OR REPLACE TABLE {table_name} AS SELECT * FROM df")
        print(f"Table {table_name} created")

# # Snowflake: specify the con info and table ame
# con = snowflake.connector.connect(
#     # check out your url: {account}.snowflakecomputing.com
#     account="",
#     user="",
#     password="",
#     warehouse="",
#     database="",
#     schema="",
# )
# # Please enter the table to stage/clean
# # Make sure it's a table (queries over view can be slow)

Table linkedin_ad_analytics_by_creative_data created
Table linkedin_campaign_history_data created
Table linkedin_ad_analytics_by_campaign_data created
Table linkedin_account_history_data created
Table linkedin_creative_history_data created
Table linkedin_campaign_group_history_data created


In [5]:
query_widget, cocoon_workflow = create_cocoon_workflow(con, para={"dbt_directory":dbt_directory,"dbt_name": dbt_name})
# query_widget, cocoon_workflow = create_cocoon_data_vault_workflow(con, para={"dbt_directory":dbt_directory,"dbt_name": dbt_name})

In [6]:
query_widget.display()

In [7]:
cocoon_workflow.start_workflow()

/usr/local/lib/python3.10/dist-packages/IPython/core/display.py:724: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


⚠️ Warning: Failed to save HTML. File already exists.
🎉 File saved successfully as ./linkedin/models/model.html
